# ทดสอบ LLM Backend API - คู่มือสำหรับ Person 4

**สำหรับ Person 4: ผู้ดูแล LLM Backend**

## เกี่ยวกับ Notebook นี้

Notebook นี้ใช้สำหรับทดสอบ:
- การเชื่อมต่อ LLM API (Google Gemini/OpenAI)
- การสร้างประโยคไทยจากคำภาษามือ
- การประมวลผลข้อมูลจาก Hand Models + Face API
- การทดสอบ Backend API โดยไม่ต้องรันเซิร์ฟเวอร์

## วิธีใช้

1. ตั้งค่า API Key ในไฟล์ .env
2. รันทุกเซลล์ตามลำดับ
3. ทดสอบการสร้างประโยค
4. ตรวจสอบผลลัพธ์

In [ ]:
# ขั้นตอนที่ 1: ตั้งค่า Environment และ API Keys

import os
from dotenv import load_dotenv

print("🔧 กำลังตั้งค่า Environment...")

# โหลดไฟล์ .env (สร้างถ้ายังไม่มี)
env_file = ".env"
if not os.path.exists(env_file):
    print("⚠️ ไม่พบไฟล์ .env - กำลังสร้างไฟล์ตัวอย่าง...")
    
    with open(env_file, 'w', encoding='utf-8') as f:
        f.write("""# Thai-HandMate Backend Configuration
# คู่มือการขอ API Key: ดู README.md

# Google Gemini API (แนะนำ - ฟรี)
GEMINI_API_KEY=your_gemini_api_key_here
GEMINI_API_BASE=https://generativelanguage.googleapis.com/v1beta/models/gemini-pro:generateContent

# OpenAI API (ทางเลือก - ต้องจ่าย)
OPENAI_API_KEY=your_openai_api_key_here
OPENAI_API_BASE=https://api.openai.com/v1/chat/completions

# ตั้งค่าโมเดล
LLM_PROVIDER=gemini
""")
    print("✅ สร้างไฟล์ .env ตัวอย่างแล้ว - กรุณาใส่ API Key ของคุณ")

# โหลดค่าตั้งค่า
load_dotenv()

# ตรวจสอบการตั้งค่า
gemini_key = os.getenv('GEMINI_API_KEY', '')
openai_key = os.getenv('OPENAI_API_KEY', '')
provider = os.getenv('LLM_PROVIDER', 'gemini')

print(f"\n📋 สถานะการตั้งค่า:")
print(f"- LLM Provider: {provider}")
print(f"- Gemini API Key: {'✅ มี' if gemini_key and gemini_key != 'your_gemini_api_key_here' else '❌ ไม่มี'}")
print(f"- OpenAI API Key: {'✅ มี' if openai_key and openai_key != 'your_openai_api_key_here' else '❌ ไม่มี'}")

if provider == 'gemini' and (not gemini_key or gemini_key == 'your_gemini_api_key_here'):
    print("\n⚠️ กรุณาใส่ Gemini API Key ในไฟล์ .env")
    print("📖 วิธีขอ API Key: ดู backend/README.md")
elif provider == 'openai' and (not openai_key or openai_key == 'your_openai_api_key_here'):
    print("\n⚠️ กรุณาใส่ OpenAI API Key ในไฟล์ .env")
else:
    print(f"\n✅ พร้อมใช้งาน {provider.upper()} API!")

In [ ]:
# Import libraries ที่จำเป็น
import json
import httpx
from typing import List

async def generate_sentences(words: List[str]) -> dict:
    """
    ฟังก์ชันสร้างประโยคไทย - เหมือนกับใน app.py
    """
    
    if not words:
        return {"error": "ต้องระบุคำอย่างน้อย 1 คำ"}
    
    words_text = " ".join(words)
    
    # ลองใช้ Typhoon API ก่อน
    if TYPHOON_API_KEY:
        try:
            sentences = await generate_with_typhoon(words)
            return {"sentences": sentences, "provider": "typhoon"}
        except Exception as e:
            print(f"[ERROR] Typhoon API ไม่สามารถใช้งานได้: {e}")
            # ถ้า API ไม่ได้ ใช้ fallback
    
    # Fallback: สร้างประโยคแบบง่าย
    fallback_sentences = generate_fallback_sentences(words_text)
    return {"sentences": fallback_sentences, "provider": "fallback"}

async def generate_with_typhoon(words: List[str]) -> List[str]:
    """สร้างประโยคด้วย Typhoon LLM"""
    
    words_text = " ".join(words)
    
    # สร้าง prompt ภาษาไทย
    system_prompt = """คุณเป็นผู้ช่วยที่เชี่ยวชาญในการสร้างประโยคภาษาไทยจากคำศัพท์ภาษามือ 
ให้สร้างประโยคที่เป็นธรรมชาติและสื่อความหมายได้ชัดเจน โดยใช้คำที่ให้มาทั้งหมดหรือส่วนใหญ่"""

    user_prompt = f"""จากคำเหล่านี้: {words_text}
กรุณาสร้างประโยคไทย 2-3 ประโยค ที่เป็นธรรมชาติและสื่อความหมายได้ดี

ตอบกลับในรูปแบบ JSON:
{{"sentences": ["ประโยคที่ 1", "ประโยคที่ 2", "ประโยคที่ 3"]}}"""

    payload = {
        "model": "typhoon-v1.5x-70b-instruct",
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        "max_tokens": 500,
        "temperature": 0.7,
        "response_format": {"type": "json_object"}
    }

    headers = {
        "Authorization": f"Bearer {TYPHOON_API_KEY}",
        "Content-Type": "application/json"
    }

    async with httpx.AsyncClient() as client:
        response = await client.post(
            TYPHOON_API_BASE,
            json=payload,
            headers=headers,
            timeout=30.0
        )
        
        if response.status_code != 200:
            raise Exception(f"API Error: {response.status_code}")
        
        data = response.json()
        content = data['choices'][0]['message']['content']
        
        # แปลง JSON response
        try:
            result = json.loads(content)
            sentences = result.get('sentences', [])
            return sentences[:3]  # เอาแค่ 3 ประโยคแรก
        except json.JSONDecodeError:
            # ถ้า JSON ไม่ถูกต้อง ใช้เนื้อหาโดยตรง
            return [content]

def generate_fallback_sentences(words_text: str) -> List[str]:
    """สร้างประโยคแบบ fallback เมื่อ API ไม่พร้อมใช้งาน"""
    
    # ประโยคพื้นฐาน
    basic = words_text
    
    # เพิ่มคำสุภาพ
    polite = f"{words_text} ครับ/ค่ะ"
    
    # เพิ่มบริบท
    contextual = f"ฉันต้องการสื่อว่า {words_text}"
    
    return [basic, polite, contextual]

print("[SUCCESS] ฟังก์ชันพร้อมใช้งาน!")

In [ ]:
# ทดสอบกับคำตัวอย่าง
import asyncio

# ตัวอย่างคำจากภาษามือไทย
test_words = ["ผม", "รัก", "คุณ"]

print(f"🧪 ทดสอบกับคำ: {test_words}")
    "print("[PROCESS] กำลังสร้างประโยค...")
",

# เรียกใช้ฟังก์ชัน
result = await generate_sentences(test_words)

print("\n[RESULT] ผลลัพธ์:")
print(f"Provider: {result.get('provider', 'unknown')}")
print("\nประโยคที่ได้:")
for i, sentence in enumerate(result.get('sentences', []), 1):
    print(f"{i}. {sentence}")

print("\n" + "="*50)

In [ ]:
# ทดสอบกับคำหลายแบบ
test_cases = [
    ["สวัสดี"],
    ["ขอบคุณ", "โอเค"],
    ["ผม", "ไป", "บ้าน"],
    ["คุณ", "สวัสดี", "ขอโทษ", "มา"]
]

print("[TEST] ทดสอบหลายกรณี:\n")

for i, words in enumerate(test_cases, 1):
    print(f"กรณีที่ {i}: {words}")
    result = await generate_sentences(words)
    
    print(f"Provider: {result.get('provider')}")
    for j, sentence in enumerate(result.get('sentences', []), 1):
        print(f"  {j}. {sentence}")
    print()

print("[SUCCESS] การทดสอบเสร็จสิ้น!")

In [ ]:
# ทดสอบ JSON output format
words = ["ผม", "รัก", "คุณ"]
result = await generate_sentences(words)

print("[FORMAT] JSON Response Format:")
print(json.dumps(result, ensure_ascii=False, indent=2))

print("\\n[SUCCESS] ระบบพร้อมใช้งาน! สามารถรัน backend server ได้แล้ว")
print("[INFO] รันด้วย: python app.py")